# Chroma | クロマ> [Chroma](https://docs.trychroma.com/getting-started)は、開発者の生産性と幸福に焦点を当てたAIネイティブのオープンソースベクターデータベースです。ChromaはApache 2.0ライセンスの下で提供されています。>> > [Chroma](https://docs.trychroma.com/getting-started) is a AI-native open-source vector database focused on developer productivity and happiness. Chroma is licensed under Apache 2.0.以下のコマンドでChromaをインストールしてください：> Install Chroma with:```shpip install chromadb```Chromaは様々なモードで動作します。LangChainと統合された各モードの例については以下を参照してください。> Chroma runs in various modes. See below for examples of each integrated with LangChain.* `in-memory` - PythonスクリプトやJupyterノートブック内で  > `in-memory` - in a python script or jupyter notebook* `インメモリで永続化` - スクリプトやノートブック内で使用し、ディスクへの保存・読み込みを行います  > `in-memory with persistance` - in a script or notebook and save/load to disk* `dockerコンテナ`内 - ローカルマシン上で動作するサーバー、またはクラウド上で  > `in a docker container` - as a server running your local machine or in the cloud他のデータベースと同様に、以下の操作ができます：> Like any other database, you can:* .add  > `.add`* `get`メソッド  > `.get`* .update  > `.update`* `upsert`は、指定されたデータが存在する場合は更新し、存在しない場合は新規に挿入する操作を行います。  > `.upsert`* .delete  > `.delete`* `peek`メソッド  > `.peek`* そして、`.query` は類似性検索を実行します。  > and `.query` runs the similarity search.完全なドキュメントは[docs](https://docs.trychroma.com/reference/Collection)でご覧いただけます。これらのメソッドに直接アクセスするには、`._collection.method()`と入力してください。> View full docs at [docs](https://docs.trychroma.com/reference/Collection). To access these methods directly, you can do `._collection.method()`

## Basic Example | 基本的な例この基本的な例では、最新の一般教書演説をチャンクに分割し、オープンソースの埋め込みモデルを用いて埋め込み、それをChromaに読み込んでからクエリを実行します。> In this basic example, we take the most recent State of the Union Address, split it into chunks, embed it using an open-source embedding model, load it into Chroma, and then query it.

In [1]:
# import
from langchain.document_loaders import TextLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

# load the document and split it into chunks
loader = TextLoader("../../modules/state_of_the_union.txt")
documents = loader.load()

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

# query it
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)

/Users/jeff/.pyenv/versions/3.10.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


## Basic Example (including saving to disk) | 基本的な例（ディスクへの保存を含む）前の例を拡張して、ディスクに保存したい場合は、Chromaクライアントを初期化して、データを保存したいディレクトリを渡すだけです。> Extending the previous example, if you want to save to disk, simply initialize the Chroma client and pass the directory where you want the data to be saved to.`注意`：Chromaはデータをディスクに自動的に保存するよう最善を尽くしていますが、複数のインメモリクライアントがお互いの作業を上書きする可能性があります。ベストプラクティスとして、ある特定のパスで一度に一つのクライアントだけが動作するようにしてください。> `Caution`: Chroma makes a best-effort to automatically save data to disk, however multiple in-memory clients can stomp each other's work. As a best practice, only have one client per path running at any given time.

In [2]:
# save to disk
db2 = Chroma.from_documents(docs, embedding_function, persist_directory="./chroma_db")
docs = db2.similarity_search(query)

# load from disk
db3 = Chroma(persist_directory="./chroma_db", embedding_function=embedding_function)
docs = db3.similarity_search(query)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


## Passing a Chroma Client into Langchain | LangchainにChroma Clientを渡すまた、Chroma Clientを作成してLangChainに渡すこともできます。これは、基盤となるデータベースへのアクセスをより容易にしたい場合に特に便利です。> You can also create a Chroma Client and pass it to LangChain. This is particularly useful if you want easier access to the underlying database.また、LangChainに使用してほしいコレクション名を指定することもできます。> You can also specify the collection name that you want LangChain to use.

In [3]:
import chromadb

persistent_client = chromadb.PersistentClient()
collection = persistent_client.get_or_create_collection("collection_name")
collection.add(ids=["1", "2", "3"], documents=["a", "b", "c"])

langchain_chroma = Chroma(
    client=persistent_client,
    collection_name="collection_name",
    embedding_function=embedding_function,
)

print("There are", langchain_chroma._collection.count(), "in the collection")

Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 1
Insert of existing embedding ID: 1
Add of existing embedding ID: 2
Insert of existing embedding ID: 2
Add of existing embedding ID: 3
Insert of existing embedding ID: 3


There are 3 in the collection


## Basic Example (using the Docker Container) | 基本的な例（Dockerコンテナの使用）また、Chroma Serverを別個のDockerコンテナで実行し、それに接続するクライアントを作成して、LangChainに渡すこともできます。> You can also run the Chroma Server in a Docker container separately, create a Client to connect to it, and then pass that to LangChain.Chromaは複数の`Collections`のドキュメントを扱う能力を持っていますが、LangChainインターフェースは一つのコレクションを期待しているため、コレクション名を指定する必要があります。LangChainが使用するデフォルトのコレクション名は「langchain」です。> Chroma has the ability to handle multiple `Collections` of documents, but the LangChain interface expects one, so we need to specify the collection name. The default collection name used by LangChain is "langchain".Dockerイメージをクローン、ビルド、そして実行する方法は以下の通りです：> Here is how to clone, build, and run the Docker Image:```shgit clone git@github.com:chroma-core/chroma.git````docker-compose.yml` ファイルを編集し、`environment` の下に `ALLOW_RESET=TRUE` を追加してください> Edit the `docker-compose.yml` file and add `ALLOW_RESET=TRUE` under `environment````yaml    ...    command: uvicorn chromadb.app:app --reload --workers 1 --host 0.0.0.0 --port 8000 --log-config log_config.yml    environment:      - IS_PERSISTENT=TRUE      - ALLOW_RESET=TRUE    ports:      - 8000:8000    ...```その後、`docker-compose up -d --build`を実行してください> Then run `docker-compose up -d --build`

In [4]:
# create the chroma client
import uuid

import chromadb
from chromadb.config import Settings

client = chromadb.HttpClient(settings=Settings(allow_reset=True))
client.reset()  # resets the database
collection = client.create_collection("my_collection")
for doc in docs:
    collection.add(
        ids=[str(uuid.uuid1())], metadatas=doc.metadata, documents=doc.page_content
    )

# tell LangChain to use our client and collection name
db4 = Chroma(
    client=client,
    collection_name="my_collection",
    embedding_function=embedding_function,
)
query = "What did the president say about Ketanji Brown Jackson"
docs = db4.similarity_search(query)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


## Update and Delete | 更新と削除実際のアプリケーションを構築する際には、データを追加するだけでなく、データの更新や削除も行いたいと思うでしょう。> While building toward a real application, you want to go beyond adding data, and also update and delete data.Chromaでは、帳簿処理を簡素化するためにユーザーに`ids`の提供を求めています。`ids`はファイル名や、`filename_paragraphNumber`のような組み合わせハッシュなどが使用できます。> Chroma has users provide `ids` to simplify the bookkeeping here. `ids` can be the name of the file, or a combined has like `filename_paragraphNumber`, etc.Chromaはこれらの操作をすべてサポートしていますが、いくつかはまだLangChainインターフェースを通じて完全に統合されている途中段階です。さらにワークフローの改善が間もなく追加される予定です。> Chroma supports all these operations - though some of them are still being integrated all the way through the LangChain interface. Additional workflow improvements will be added soon.以下は、様々な操作を行う方法を示す基本的な例です：> Here is a basic example showing how to do various operations:

In [5]:
# create simple ids
ids = [str(i) for i in range(1, len(docs) + 1)]

# add data
example_db = Chroma.from_documents(docs, embedding_function, ids=ids)
docs = example_db.similarity_search(query)
print(docs[0].metadata)

# update the metadata for a document
docs[0].metadata = {
    "source": "../../modules/state_of_the_union.txt",
    "new_value": "hello world",
}
example_db.update_document(ids[0], docs[0])
print(example_db._collection.get(ids=[ids[0]]))

# delete the last document
print("count before", example_db._collection.count())
example_db._collection.delete(ids=[ids[-1]])
print("count after", example_db._collection.count())

{'source': '../../../state_of_the_union.txt'}
{'ids': ['1'], 'embeddings': None, 'metadatas': [{'new_value': 'hello world', 'source': '../../../state_of_the_union.txt'}], 'documents': ['Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.']}
count before 46
c

## Use OpenAI Embeddings | OpenAIの埋め込みを使用する多くの人がOpenAIの埋め込み機能を好んで使用しています。設定方法は以下の通りです。> Many people like to use OpenAIEmbeddings, here is how to set that up.

In [6]:
# get a token: https://platform.openai.com/account/api-keys

from getpass import getpass

from langchain.embeddings.openai import OpenAIEmbeddings

OPENAI_API_KEY = getpass()

In [7]:
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [8]:
embeddings = OpenAIEmbeddings()
new_client = chromadb.EphemeralClient()
openai_lc_client = Chroma.from_documents(
    docs, embeddings, client=new_client, collection_name="openai_collection"
)

query = "What did the president say about Ketanji Brown Jackson"
docs = openai_lc_client.similarity_search(query)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


***## Other Information | その他の情報

### Similarity search with score | スコア付き類似性検索

返される距離スコアはコサイン距離です。したがって、スコアが低いほど良いです。> The returned distance score is cosine distance. Therefore, a lower score is better.

In [9]:
docs = db.similarity_search_with_score(query)

In [10]:
docs[0]

(Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'source': '../../../state_of_the_union.txt'}),
 1.1972057819366455)

### Retriever options | レトリバーのオプションこのセクションでは、Chromaをリトリーバーとして使用するさまざまな方法について説明します。> This section goes over different options for how to use Chroma as a retriever.#### MMR | MMR検索オブジェクトで類似性検索を使用することに加えて、`mmr`を使用することもできます。> In addition to using similarity search in the retriever object, you can also use `mmr`.

In [11]:
retriever = db.as_retriever(search_type="mmr")

In [12]:
retriever.get_relevant_documents(query)[0]

Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'source': '../../../state_of_the_union.txt'})

### Filtering on metadata | メタデータによるフィルタリングコレクションを操作する前に、それを絞り込むことが役立つことがあります。> It can be helpful to narrow down the collection before working with it.例えば、getメソッドを使用してメタデータを使ってコレクションをフィルタリングすることができます。> For example, collections can be filtered on metadata using the get method.

In [13]:
# filter collection for updated source
example_db.get(where={"source": "some_other_source"})

{'ids': [], 'embeddings': None, 'metadatas': [], 'documents': []}